# Senior Project Notebook  
In the first phase I have downloaded the matched dataset from the Lakh Midi Dataset and cleared the songs with duplicate midi files after that I have run a web scraper for spotify api to get the features about the songs from spotify like tempo, danceability and so on. I have tried to match artists and songs that were available in spotify. After removing all the duplicates and unmatched songs 3800 songs are left for training
Songs are in the clear_midi file,where the folder name specifies the artist name and the file name specifies the artist name. The scraper I have used stores the data in a file called statistics.csv


In [18]:
from music21 import converter, instrument, note, chord
import matplotlib
import pandas as pd
import numpy as np
from math import ceil
note_dict = {
        'C':0,
        'D':2,
        'E':4,
        'F':5,
        'G':7,
        'A':9,
        'B':11
    }

In [2]:
# Function to get numerical note value from text
def get_notes(text):
    val = (int(text[-1]) - 1) * 12
    val = val + note_dict[text[0]]
    if text[1] == '-':
        val = val -1
    elif text[1] == '#':
        val = val + 1
    return val

### Input data construction  
Here we construct a matrix with rows as the duration * 4 to take each row as a quarter note  
And the columns consists of 73 notes from C1 to C7 and additional duration feature

In [51]:
def midi_to_input(artist, song, key):
    midi = converter.parse('./clean_midi/' + artist+'/'+ song + '.mid')
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts: 
        # get the track with the most notes from the instruments
        notes_to_parse = max(parts.parts, key=lambda p: p.__len__()).flat.notes
    else: 
        #single instrument
        notes_to_parse = midi.flat.notes
    transposed = transpose(notes_to_parse, key, notes_to_parse.analyze('ambitus').noteStart, notes_to_parse.analyze('ambitus').noteEnd)
    print("Range before transpose: ",notes_to_parse.analyze('ambitus').noteEnd,notes_to_parse.analyze('ambitus').noteStart)
    print("Range after  transpose: ",transposed.analyze('ambitus').noteEnd,transposed.analyze('ambitus').noteStart)
    duration = notes_to_parse.duration.quarterLength
    notes = np.zeros((ceil(duration*4), 74))
    for element in transposed:
        if isinstance(element, note.Note):
            timestep = int(round(element.offset*4)) 
            notes[timestep, get_notes(element.pitch.nameWithOctave)] = element.volume.velocityScalar
            notes[timestep, 73] = max(notes[timestep, 73], element.duration.quarterLength)
        elif isinstance(element, chord.Chord):
            timestep = int(round(element.offset*4)) 
            for part in element:
                notes[timestep, get_notes(part.pitch.nameWithOctave)] = part.volume.velocityScalar
            notes[timestep, 73] = max(notes[timestep, 73], element.duration.quarterLength)     
    return notes

In [47]:
def transpose(note_stream, key,spectrumStart,spectrumEnd):
    start = get_notes(spectrumStart.nameWithOctave)
    if (key < 6) and (key<=start):
        offset = -1 * key 
    else:
        offset = 12 - key
    return note_stream.transpose(offset)

In [48]:
songs = pd.read_csv('./statistics.csv')
selected_song = songs.iloc[2671]

In [52]:
input = midi_to_input(selected_song.artist_name, selected_song.song_name, selected_song.key)

B-6 C2
G#6 B-1


In [50]:
selected_song.key

2